In [1]:
%load_ext autotime

In [2]:
# Importar librerias
import pandas as pd
import numpy as np

import zipfile
from datetime import date
from sklearn import metrics
from sklearn.metrics import accuracy_score


from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials

from numpy.testing import assert_allclose
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta

import os
from pathlib import Path


Using TensorFlow backend.


time: 7.93 s


In [3]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)

time: 2 ms


In [4]:
exp_name = 'Embed_promedio'
folder = 'Resultados/' + exp_name
my_file = Path(folder)
if os.path.exists(my_file):
    print('already exists')
else:
    os.makedirs(folder)
    


already exists
time: 136 ms


In [5]:
ch_folder = folder + '/Checkpoints'
my_file = Path(ch_folder)
if os.path.exists(my_file):
    print('already exists')
else:
    os.makedirs(ch_folder)

already exists
time: 114 ms


### Read datasets

In [6]:
#Importar los datasets
url_embeddings_average_individual = zipfile.ZipFile('../Data/embeddings_average_individual.zip')
url_embeddings_sum_individual = zipfile.ZipFile('../Data/embeddings_sum_individual.zip')

embeddings_average_individual = pd.read_csv(url_embeddings_average_individual.open('embeddings_average_individual.csv'), index_col = 0)
embeddings_sum_individual =pd.read_csv(url_embeddings_sum_individual.open('embeddings_sum_individual.csv'), index_col = 0)

embeddings_average_individual['Date'] =  pd.to_datetime(embeddings_average_individual['Date'], format='%Y-%m-%d')
# embeddings_average_individual.sort_values('Date', inplace=True)
# embeddings_average_individual.reset_index(drop = True , inplace=True)

embeddings_average_individual.tail(2)

,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,Label,Date,Top
49716,-0.082072,0.099915,-0.015503,0.115560,-0.072611,0.070435,0.042613,-0.041026,0.012126,-0.020508,...,-0.062215,-0.078471,0.046008,0.005717,-0.071452,0.122559,0.07622,0,2008-08-08,24
49717,-0.064514,0.013916,-0.028976,0.058716,-0.078369,-0.057312,-0.077515,-0.234467,0.050751,0.020508,...,-0.010498,0.081284,0.040283,-0.108978,0.033783,0.028870,0.03418,0,2008-08-08,25


time: 12.6 s


### Embedding Promedio

In [7]:
# Selecciono la fecha para la cual hago el corte de train y test
training_end = pd.to_datetime("2013-12-31")
num_training = len(embeddings_average_individual[(embeddings_average_individual["Date"]) <= training_end])

time: 90 ms


In [8]:
# Selecciono el archivo con el que se corre el modelo
data = embeddings_average_individual[embeddings_average_individual['Date']<='2014-12-31']

time: 71 ms


In [9]:
# Se separa en train y test
x_train = data.drop(["Top","Label", "Date"], axis=1)[:num_training]
x_test = data.drop(["Top",'Label', 'Date'], axis=1)[num_training:]
y_train = data["Label"].values[:num_training]
y_test = data["Label"].values[num_training:]


x_train_array = x_train.to_numpy()
reshape_x_train = x_train_array.reshape(len(x_train), 1, 300)
reshape_x_train.shape

x_test_array = x_test.to_numpy()
reshape_x_test = x_test_array.reshape(len(x_test), 1, 300)
reshape_x_test.shape

(6300, 1, 300)

time: 268 ms


### Definir espacio de busqueda

In [10]:
space = {
    'units1': hp.choice('units1', [10, 64, 128, 256, 512]),
    'units2': hp.choice('units2', [10, 64, 128, 256, 512]),
                 
    'dropout1': hp.choice('dropout1', [0.2,0.3,0.1]),
    
    'batch_size' : hp.choice('batch_size', [128,256,512]),
    'nb_epochs' : hp.choice('nb_epochs', [50]),

    'optimizer':  hp.choice('optimizer', [ 'adam','adadelta']),   
    'activation': 'relu'    
    }

time: 149 ms


### Definir busqueda bayesiana

In [11]:
#Objective function that hyperopt will minimize
from timeit import default_timer as timer

def objective(params):
    
    import ml_metrics

    
    start = timer()
    print ('Params testing: ', params)
    print ('\n ')
    
    model = Sequential()
    model.add(LSTM(params['units1'], input_shape=(1,300), return_sequences=True))
    model.add(Dropout(params['dropout1']))
    model.add(LSTM(params['units2'], return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))
    #model.compile(loss='mean_squared_error', optimizer='adam')
    # compile the model
    model.compile(optimizer=params['optimizer'], loss='binary_crossentropy', metrics=['accuracy'])


    
    #includes the call back object
    model.fit(reshape_x_train, y_train, epochs=params['nb_epochs'], batch_size=params['batch_size'],
              verbose = 0, validation_data=(reshape_x_test, y_test))
     
    #predict the test set 
    score, acc = model.evaluate(reshape_x_test, y_test, verbose=0)
    
    run_time = timer() - start
    
    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([-acc, params, score, run_time])
    of_connection.close()
    
    
    print('Test accuracy:', acc)
 
    return {'loss': -acc, 'status': STATUS_OK, 'train_time': run_time,}



time: 125 ms


### Almacenar resultados de cada iteración

In [12]:
from hyperopt import tpe

tpe_algorithm = tpe.suggest

time: 73.5 ms


In [13]:
import csv

from hyperopt import Trials

bayes_trials = Trials()

# File to save first results
out_file = folder + '/gbm_results.csv'
of_connection = open(out_file, 'w')

writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['loss', 'params', 'score','time'])
of_connection.close()

time: 114 ms


### Lanzar optimización

In [14]:
trials = Trials()


# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = 20, trials = bayes_trials,
            verbose = 1, rstate= np.random.RandomState(50))


Params testing:                                                                                                        
{'activation': 'relu', 'batch_size': 512, 'dropout1': 0.3, 'nb_epochs': 50, 'optimizer': 'adadelta', 'units1': 128, 'units2': 128}
Test accuracy:                                                                                                         
0.4920634925365448                                                                                                     
Params testing:                                                                                                        
{'activation': 'relu', 'batch_size': 256, 'dropout1': 0.3, 'nb_epochs': 50, 'optimizer': 'adam', 'units1': 64, 'units2': 10}
Test accuracy:                                                                                                         
0.5046031475067139                                                                                                     
Params testing:         

0.4920634925365448                                                                                                     
Params testing:                                                                                                        
{'activation': 'relu', 'batch_size': 512, 'dropout1': 0.3, 'nb_epochs': 50, 'optimizer': 'adadelta', 'units1': 256, 'units2': 64}
Test accuracy:                                                                                                         
0.4920634925365448                                                                                                     
Params testing:                                                                                                        
{'activation': 'relu', 'batch_size': 512, 'dropout1': 0.3, 'nb_epochs': 50, 'optimizer': 'adam', 'units1': 256, 'units2': 10}
Test accuracy:                                                                                                         
0.49968254566192627     

### Exportar bayesiana, por si quisiera retomar donde queda

In [15]:
import pickle
pickle.dump(bayes_trials, open(folder + '/trials.pkl', 'wb'))

time: 4 ms


### Leer mejores parametros

In [16]:
# Sort the trials with lowest loss (highest AUC) first
bayes_trials_results  = sorted(bayes_trials.results, key = lambda x: x['loss'])
bayes_trials_results [:2]

[{'loss': -0.5073015689849854,
  'status': 'ok',
  'train_time': 363.16455289999976},
 {'loss': -0.506507933139801,
  'status': 'ok',
  'train_time': 197.95745129999978}]

time: 100 ms


In [17]:
results = pd.read_csv(folder + '/gbm_results.csv')

# Sort with best scores on top and reset index for slicing
results.sort_values('loss', ascending = True, inplace = True)
results.reset_index(inplace = True, drop = True)
results.head()

,loss,params,score,time
0,-0.507302,"{'activation': 'relu', 'batch_size': 512, 'dro...",NaN,363.164553
1,-0.506508,"{'activation': 'relu', 'batch_size': 512, 'dro...",NaN,188.239897
2,-0.506508,"{'activation': 'relu', 'batch_size': 128, 'dro...",NaN,197.957451
3,-0.506190,"{'activation': 'relu', 'batch_size': 256, 'dro...",NaN,391.347929
4,-0.504603,"{'activation': 'relu', 'batch_size': 256, 'dro...",NaN,62.543316


time: 130 ms


In [18]:
import ast

# Convert from a string to a dictionary
ast.literal_eval(results.loc[0, 'params'])

{'activation': 'relu',
 'batch_size': 512,
 'dropout1': 0.2,
 'nb_epochs': 50,
 'optimizer': 'adam',
 'units1': 64,
 'units2': 512}

time: 77.7 ms


In [19]:
# Extract the ideal number of estimators and hyperparameters
best_bayes_params = ast.literal_eval(results.loc[0, 'params']).copy()
best_bayes_params

{'activation': 'relu',
 'batch_size': 512,
 'dropout1': 0.2,
 'nb_epochs': 50,
 'optimizer': 'adam',
 'units1': 64,
 'units2': 512}

time: 131 ms


### Definir datasets de testeo

In [20]:
# Selecciono la fecha para la cual hago el corte de train y test
training_end = pd.to_datetime("2014-12-31")
num_training = len(embeddings_average_individual[(embeddings_average_individual["Date"]) <= training_end])

time: 219 ms


In [21]:
# Selecciono el archivo con el que se corre el modelo
data = embeddings_average_individual

time: 134 ms


In [22]:
# Se separa en train y test
x_train = data.drop(["Top","Label", "Date"], axis=1)[:num_training]
x_test = data.drop(["Top",'Label', 'Date'], axis=1)[num_training:]
y_train = data["Label"].values[:num_training]
y_test = data["Label"].values[num_training:]


x_train_array = x_train.to_numpy()
reshape_x_train = x_train_array.reshape(len(x_train), 1, 300)
reshape_x_train.shape

x_test_array = x_test.to_numpy()
reshape_x_test = x_test_array.reshape(len(x_test), 1, 300)
reshape_x_test.shape

(9450, 1, 300)

time: 278 ms


In [23]:
model = Sequential()
model.add(LSTM(best_bayes_params['units1'], input_shape=(1,300), return_sequences=True))
model.add(Dropout(best_bayes_params['dropout1']))
model.add(LSTM(best_bayes_params['units2'], return_sequences=False))
model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='mean_squared_error', optimizer='adam')
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


#model.compile(loss='mean_squared_error', optimizer='adam')
# compile the model
model.compile(loss='binary_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])


# define the checkpoint
filepath= ch_folder + "/word2vec-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

time: 1.11 s


In [24]:
# fit the model
model.fit(reshape_x_train, y_train,
          epochs=best_bayes_params['nb_epochs'], 
          batch_size=best_bayes_params['batch_size'], callbacks=callbacks_list)


model.save(folder + '/keras_model.h5')
model.summary()

Epoch 1/50
40268/40268 [==============================] - ETA: 1:06 - loss: 0.6931 - accuracy: 0.51 - ETA: 38s - loss: 0.6928 - accuracy: 0.5410 - ETA: 28s - loss: 0.6928 - accuracy: 0.534 - ETA: 23s - loss: 0.6927 - accuracy: 0.532 - ETA: 20s - loss: 0.6926 - accuracy: 0.531 - ETA: 18s - loss: 0.6923 - accuracy: 0.534 - ETA: 16s - loss: 0.6920 - accuracy: 0.537 - ETA: 15s - loss: 0.6918 - accuracy: 0.537 - ETA: 14s - loss: 0.6915 - accuracy: 0.539 - ETA: 13s - loss: 0.6914 - accuracy: 0.539 - ETA: 12s - loss: 0.6915 - accuracy: 0.537 - ETA: 12s - loss: 0.6913 - accuracy: 0.538 - ETA: 11s - loss: 0.6908 - accuracy: 0.540 - ETA: 11s - loss: 0.6908 - accuracy: 0.540 - ETA: 10s - loss: 0.6905 - accuracy: 0.541 - ETA: 10s - loss: 0.6906 - accuracy: 0.541 - ETA: 10s - loss: 0.6912 - accuracy: 0.538 - ETA: 9s - loss: 0.6910 - accuracy: 0.538 - ETA: 9s - loss: 0.6906 - accuracy: 0.54 - ETA: 9s - loss: 0.6908 - accuracy: 0.53 - ETA: 8s - loss: 0.6909 - accuracy: 0.53 - ETA: 8s - loss: 0.6908 -

40268/40268 [==============================] - ETA: 7s - loss: 0.6883 - accuracy: 0.56 - ETA: 8s - loss: 0.6906 - accuracy: 0.53 - ETA: 8s - loss: 0.6917 - accuracy: 0.52 - ETA: 8s - loss: 0.6906 - accuracy: 0.52 - ETA: 8s - loss: 0.6907 - accuracy: 0.52 - ETA: 7s - loss: 0.6904 - accuracy: 0.52 - ETA: 7s - loss: 0.6905 - accuracy: 0.52 - ETA: 7s - loss: 0.6907 - accuracy: 0.52 - ETA: 7s - loss: 0.6906 - accuracy: 0.53 - ETA: 7s - loss: 0.6907 - accuracy: 0.52 - ETA: 7s - loss: 0.6904 - accuracy: 0.53 - ETA: 7s - loss: 0.6901 - accuracy: 0.53 - ETA: 7s - loss: 0.6902 - accuracy: 0.53 - ETA: 7s - loss: 0.6903 - accuracy: 0.53 - ETA: 7s - loss: 0.6899 - accuracy: 0.53 - ETA: 6s - loss: 0.6899 - accuracy: 0.53 - ETA: 6s - loss: 0.6899 - accuracy: 0.53 - ETA: 6s - loss: 0.6897 - accuracy: 0.53 - ETA: 6s - loss: 0.6897 - accuracy: 0.53 - ETA: 6s - loss: 0.6896 - accuracy: 0.53 - ETA: 6s - loss: 0.6894 - accuracy: 0.53 - ETA: 6s - loss: 0.6898 - accuracy: 0.53 - ETA: 6s - loss: 0.6896 - accu

40268/40268 [==============================] - ETA: 8s - loss: 0.6889 - accuracy: 0.51 - ETA: 8s - loss: 0.6888 - accuracy: 0.52 - ETA: 8s - loss: 0.6893 - accuracy: 0.52 - ETA: 8s - loss: 0.6892 - accuracy: 0.52 - ETA: 8s - loss: 0.6892 - accuracy: 0.53 - ETA: 8s - loss: 0.6900 - accuracy: 0.52 - ETA: 8s - loss: 0.6901 - accuracy: 0.52 - ETA: 8s - loss: 0.6895 - accuracy: 0.52 - ETA: 8s - loss: 0.6893 - accuracy: 0.53 - ETA: 8s - loss: 0.6889 - accuracy: 0.53 - ETA: 7s - loss: 0.6886 - accuracy: 0.53 - ETA: 7s - loss: 0.6888 - accuracy: 0.53 - ETA: 7s - loss: 0.6894 - accuracy: 0.53 - ETA: 7s - loss: 0.6898 - accuracy: 0.53 - ETA: 7s - loss: 0.6896 - accuracy: 0.53 - ETA: 7s - loss: 0.6896 - accuracy: 0.53 - ETA: 7s - loss: 0.6895 - accuracy: 0.53 - ETA: 7s - loss: 0.6897 - accuracy: 0.53 - ETA: 7s - loss: 0.6895 - accuracy: 0.53 - ETA: 6s - loss: 0.6894 - accuracy: 0.53 - ETA: 6s - loss: 0.6892 - accuracy: 0.53 - ETA: 6s - loss: 0.6891 - accuracy: 0.53 - ETA: 6s - loss: 0.6889 - accu

40268/40268 [==============================] - ETA: 8s - loss: 0.6816 - accuracy: 0.57 - ETA: 8s - loss: 0.6828 - accuracy: 0.56 - ETA: 8s - loss: 0.6855 - accuracy: 0.55 - ETA: 8s - loss: 0.6852 - accuracy: 0.54 - ETA: 8s - loss: 0.6838 - accuracy: 0.55 - ETA: 8s - loss: 0.6841 - accuracy: 0.55 - ETA: 8s - loss: 0.6848 - accuracy: 0.55 - ETA: 7s - loss: 0.6861 - accuracy: 0.55 - ETA: 7s - loss: 0.6852 - accuracy: 0.55 - ETA: 7s - loss: 0.6854 - accuracy: 0.55 - ETA: 7s - loss: 0.6857 - accuracy: 0.55 - ETA: 7s - loss: 0.6852 - accuracy: 0.55 - ETA: 7s - loss: 0.6853 - accuracy: 0.55 - ETA: 7s - loss: 0.6855 - accuracy: 0.55 - ETA: 7s - loss: 0.6857 - accuracy: 0.55 - ETA: 7s - loss: 0.6855 - accuracy: 0.55 - ETA: 7s - loss: 0.6857 - accuracy: 0.55 - ETA: 6s - loss: 0.6860 - accuracy: 0.55 - ETA: 6s - loss: 0.6859 - accuracy: 0.55 - ETA: 6s - loss: 0.6860 - accuracy: 0.54 - ETA: 6s - loss: 0.6857 - accuracy: 0.55 - ETA: 6s - loss: 0.6856 - accuracy: 0.55 - ETA: 6s - loss: 0.6856 - accu

40268/40268 [==============================] - ETA: 8s - loss: 0.6809 - accuracy: 0.57 - ETA: 8s - loss: 0.6835 - accuracy: 0.56 - ETA: 8s - loss: 0.6842 - accuracy: 0.56 - ETA: 7s - loss: 0.6854 - accuracy: 0.55 - ETA: 7s - loss: 0.6864 - accuracy: 0.53 - ETA: 7s - loss: 0.6875 - accuracy: 0.53 - ETA: 7s - loss: 0.6877 - accuracy: 0.53 - ETA: 7s - loss: 0.6876 - accuracy: 0.53 - ETA: 7s - loss: 0.6878 - accuracy: 0.53 - ETA: 7s - loss: 0.6871 - accuracy: 0.54 - ETA: 7s - loss: 0.6869 - accuracy: 0.54 - ETA: 7s - loss: 0.6868 - accuracy: 0.54 - ETA: 7s - loss: 0.6870 - accuracy: 0.54 - ETA: 6s - loss: 0.6865 - accuracy: 0.54 - ETA: 6s - loss: 0.6859 - accuracy: 0.54 - ETA: 6s - loss: 0.6852 - accuracy: 0.54 - ETA: 6s - loss: 0.6855 - accuracy: 0.54 - ETA: 6s - loss: 0.6854 - accuracy: 0.55 - ETA: 6s - loss: 0.6858 - accuracy: 0.54 - ETA: 6s - loss: 0.6858 - accuracy: 0.54 - ETA: 6s - loss: 0.6858 - accuracy: 0.54 - ETA: 6s - loss: 0.6856 - accuracy: 0.55 - ETA: 6s - loss: 0.6853 - accu

...

40268/40268 [==============================] - ETA: 8s - loss: 0.5973 - accuracy: 0.67 - ETA: 8s - loss: 0.6061 - accuracy: 0.66 - ETA: 9s - loss: 0.6254 - accuracy: 0.63 - ETA: 8s - loss: 0.6192 - accuracy: 0.64 - ETA: 8s - loss: 0.6124 - accuracy: 0.64 - ETA: 8s - loss: 0.6118 - accuracy: 0.64 - ETA: 8s - loss: 0.6150 - accuracy: 0.64 - ETA: 8s - loss: 0.6122 - accuracy: 0.64 - ETA: 8s - loss: 0.6111 - accuracy: 0.65 - ETA: 8s - loss: 0.6099 - accuracy: 0.65 - ETA: 8s - loss: 0.6095 - accuracy: 0.65 - ETA: 8s - loss: 0.6108 - accuracy: 0.64 - ETA: 7s - loss: 0.6108 - accuracy: 0.64 - ETA: 7s - loss: 0.6122 - accuracy: 0.64 - ETA: 7s - loss: 0.6131 - accuracy: 0.64 - ETA: 7s - loss: 0.6141 - accuracy: 0.64 - ETA: 7s - loss: 0.6151 - accuracy: 0.64 - ETA: 7s - loss: 0.6152 - accuracy: 0.64 - ETA: 7s - loss: 0.6152 - accuracy: 0.64 - ETA: 7s - loss: 0.6146 - accuracy: 0.64 - ETA: 7s - loss: 0.6151 - accuracy: 0.64 - ETA: 6s - loss: 0.6166 - accuracy: 0.64 - ETA: 6s - loss: 0.6170 - accu

40268/40268 [==============================] - ETA: 8s - loss: 0.5980 - accuracy: 0.66 - ETA: 8s - loss: 0.6074 - accuracy: 0.66 - ETA: 8s - loss: 0.6095 - accuracy: 0.65 - ETA: 8s - loss: 0.6103 - accuracy: 0.64 - ETA: 8s - loss: 0.6088 - accuracy: 0.64 - ETA: 8s - loss: 0.6075 - accuracy: 0.65 - ETA: 8s - loss: 0.6072 - accuracy: 0.65 - ETA: 7s - loss: 0.6084 - accuracy: 0.65 - ETA: 7s - loss: 0.6099 - accuracy: 0.65 - ETA: 7s - loss: 0.6124 - accuracy: 0.64 - ETA: 7s - loss: 0.6136 - accuracy: 0.64 - ETA: 7s - loss: 0.6126 - accuracy: 0.64 - ETA: 7s - loss: 0.6123 - accuracy: 0.64 - ETA: 7s - loss: 0.6113 - accuracy: 0.65 - ETA: 7s - loss: 0.6099 - accuracy: 0.65 - ETA: 6s - loss: 0.6095 - accuracy: 0.65 - ETA: 6s - loss: 0.6107 - accuracy: 0.65 - ETA: 6s - loss: 0.6106 - accuracy: 0.65 - ETA: 6s - loss: 0.6104 - accuracy: 0.65 - ETA: 6s - loss: 0.6097 - accuracy: 0.65 - ETA: 6s - loss: 0.6094 - accuracy: 0.65 - ETA: 6s - loss: 0.6093 - accuracy: 0.65 - ETA: 6s - loss: 0.6093 - accu

40268/40268 [==============================] - ETA: 9s - loss: 0.5979 - accuracy: 0.67 - ETA: 9s - loss: 0.5976 - accuracy: 0.67 - ETA: 9s - loss: 0.6014 - accuracy: 0.67 - ETA: 9s - loss: 0.6056 - accuracy: 0.66 - ETA: 9s - loss: 0.6046 - accuracy: 0.66 - ETA: 9s - loss: 0.6040 - accuracy: 0.66 - ETA: 9s - loss: 0.6024 - accuracy: 0.66 - ETA: 9s - loss: 0.6004 - accuracy: 0.66 - ETA: 9s - loss: 0.6039 - accuracy: 0.66 - ETA: 9s - loss: 0.6039 - accuracy: 0.66 - ETA: 9s - loss: 0.6038 - accuracy: 0.66 - ETA: 9s - loss: 0.6028 - accuracy: 0.66 - ETA: 8s - loss: 0.6012 - accuracy: 0.66 - ETA: 8s - loss: 0.6028 - accuracy: 0.66 - ETA: 8s - loss: 0.6042 - accuracy: 0.66 - ETA: 8s - loss: 0.6019 - accuracy: 0.66 - ETA: 8s - loss: 0.6015 - accuracy: 0.66 - ETA: 8s - loss: 0.6015 - accuracy: 0.66 - ETA: 8s - loss: 0.6022 - accuracy: 0.66 - ETA: 8s - loss: 0.6041 - accuracy: 0.66 - ETA: 7s - loss: 0.6033 - accuracy: 0.66 - ETA: 7s - loss: 0.6023 - accuracy: 0.66 - ETA: 7s - loss: 0.6017 - accu

40268/40268 [==============================] - ETA: 8s - loss: 0.5940 - accuracy: 0.66 - ETA: 8s - loss: 0.5970 - accuracy: 0.66 - ETA: 8s - loss: 0.5982 - accuracy: 0.66 - ETA: 8s - loss: 0.6007 - accuracy: 0.66 - ETA: 8s - loss: 0.5994 - accuracy: 0.66 - ETA: 8s - loss: 0.6019 - accuracy: 0.65 - ETA: 8s - loss: 0.6002 - accuracy: 0.65 - ETA: 8s - loss: 0.5994 - accuracy: 0.65 - ETA: 8s - loss: 0.5991 - accuracy: 0.65 - ETA: 7s - loss: 0.5960 - accuracy: 0.65 - ETA: 7s - loss: 0.5939 - accuracy: 0.66 - ETA: 7s - loss: 0.5942 - accuracy: 0.66 - ETA: 7s - loss: 0.5944 - accuracy: 0.66 - ETA: 7s - loss: 0.5937 - accuracy: 0.66 - ETA: 7s - loss: 0.5953 - accuracy: 0.66 - ETA: 7s - loss: 0.5960 - accuracy: 0.65 - ETA: 7s - loss: 0.5978 - accuracy: 0.65 - ETA: 7s - loss: 0.5985 - accuracy: 0.65 - ETA: 7s - loss: 0.5978 - accuracy: 0.65 - ETA: 6s - loss: 0.5977 - accuracy: 0.66 - ETA: 6s - loss: 0.5978 - accuracy: 0.65 - ETA: 6s - loss: 0.5986 - accuracy: 0.65 - ETA: 6s - loss: 0.5983 - accu

40268/40268 [==============================] - ETA: 8s - loss: 0.5831 - accuracy: 0.69 - ETA: 8s - loss: 0.5774 - accuracy: 0.68 - ETA: 8s - loss: 0.5789 - accuracy: 0.67 - ETA: 8s - loss: 0.5829 - accuracy: 0.67 - ETA: 8s - loss: 0.5808 - accuracy: 0.67 - ETA: 8s - loss: 0.5791 - accuracy: 0.67 - ETA: 8s - loss: 0.5765 - accuracy: 0.67 - ETA: 7s - loss: 0.5750 - accuracy: 0.68 - ETA: 7s - loss: 0.5796 - accuracy: 0.67 - ETA: 7s - loss: 0.5814 - accuracy: 0.67 - ETA: 7s - loss: 0.5805 - accuracy: 0.67 - ETA: 7s - loss: 0.5807 - accuracy: 0.67 - ETA: 7s - loss: 0.5826 - accuracy: 0.67 - ETA: 7s - loss: 0.5842 - accuracy: 0.67 - ETA: 7s - loss: 0.5862 - accuracy: 0.67 - ETA: 7s - loss: 0.5883 - accuracy: 0.66 - ETA: 7s - loss: 0.5881 - accuracy: 0.66 - ETA: 7s - loss: 0.5886 - accuracy: 0.66 - ETA: 6s - loss: 0.5883 - accuracy: 0.66 - ETA: 6s - loss: 0.5887 - accuracy: 0.66 - ETA: 6s - loss: 0.5891 - accuracy: 0.66 - ETA: 6s - loss: 0.5890 - accuracy: 0.66 - ETA: 6s - loss: 0.5885 - accu

In [25]:
# evaluate the model
loss, accuracy = model.evaluate(reshape_x_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy))

Accuracy: 0.497989
time: 1.7 s


In [26]:
%%javascript
Jupyter.notebook.save_checkpoint()

<IPython.core.display.Javascript object>

time: 28 ms


In [27]:
from time import sleep
sleep(60)

time: 1min


In [28]:
from shutil import copyfile
copyfile('RNN_Model_Base.ipynb', folder + '/RNN_Model_Base.ipynb' )

'Resultados/Embed_promedio/RNN_Model_Base.ipynb'

time: 7.02 ms


### Embedding Suma

In [29]:
# # Selecciono el archivo con el que se corre el modelo
# data = embeddings_sum_individual

time: 278 ms


In [30]:
# # Se separa en train y test
# x_train = data.drop([data.columns[0],"Top","Label", "Date"], axis=1)[:num_training]
# x_test = data.drop([data.columns[0],"Top",'Label', 'Date'], axis=1)[num_training:]
# y_train = data["Label"].values[:num_training]
# y_test = data["Label"].values[num_training:]

time: 95 ms


In [31]:
# x_train_array = x_train.to_numpy()
# reshape_x_train = x_train_array.reshape(40268, 1, 300)
# reshape_x_train.shape

time: 240 ms


In [32]:
# x_test_array = x_test.to_numpy()
# reshape_x_test = x_test_array.reshape(9450, 1, 300)
# reshape_x_test.shape

time: 147 ms


In [33]:
# from numpy.testing import assert_allclose
# from keras.models import Sequential, load_model
# from keras.layers import LSTM, Dropout, Dense
# from keras.callbacks import ModelCheckpoint

time: 103 ms


In [34]:
# n_units = 10

# model = Sequential()
# model.add(LSTM(n_units, input_shape=(1,300), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(n_units, return_sequences=False))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='mean_squared_error', optimizer='adam')
# # compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# # define the checkpoint
# filepath="word2vec-{epoch:02d}-{loss:.4f}.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

time: 105 ms


In [35]:
# # fit the model
# model.fit(reshape_x_train, y_train, epochs=50, batch_size=500, callbacks=callbacks_list)
# model.summary()

time: 141 ms


In [36]:
# # evaluate the model
# loss, accuracy = model.evaluate(reshape_x_test, y_test, verbose=0)
# print('Accuracy: %f' % (accuracy))

time: 161 ms
